In [65]:
import pandas as pd
import numpy as np
from catboost import CatBoostRegressor

In [66]:
data = pd.read_csv("complete_dataset_gpmt.csv")
data.head()

,Unnamed: 0,date,companyName,currency,language,countryName,distributorName,gender,modelNumber,description,...,consumerRating,url,division,ageGroup,subBrand,colors,currentPrice,ped,discountedSince,technologies
0,0,8/8/2018,adidas,EUR,es-ES,ESP,Zalando,female,AD111A0GU,"Material exterior: Piel/tela, Material interio...",...,4.82,https://www.zalando.es/adidas-originals-zapati...,Shoes,adults,adidas Originals,vapour grey/footwear white,67.45,NaN,11/13/2017,NaN
1,0,8/8/2018,adidas,EUR,es-ES,ESP,Zalando,female,AD111A0GV,"Material exterior: Piel/tela, Material interio...",...,4.80,https://www.zalando.es/adidas-originals-zapati...,Shoes,adults,adidas Originals,clear brown/ice pink/light brown,79.95,NaN,2/5/2018,NaN
2,0,8/8/2018,adidas,EUR,es-ES,ESP,Zalando,female,AD111A0GW,"Material exterior: Fibra sintética/tela, Mater...",...,4.89,https://www.zalando.es/adidas-originals-zapati...,Shoes,adults,adidas Originals,raw pink/footwear white,125.95,NaN,7/30/2018,NaN
3,0,8/8/2018,adidas,EUR,es-ES,ESP,Zalando,female,AD111A0H6,"Material exterior: Piel/piel de imitación, Mat...",...,5.00,https://www.zalando.es/adidas-originals-gazell...,Shoes,adults,adidas Originals,vintage white/orctin,89.95,NaN,3/19/2018,NaN
4,0,8/8/2018,adidas,EUR,es-ES,ESP,Zalando,female,AD111A0H7,"Material exterior: Piel/piel de imitación, Mat...",...,4.84,https://www.zalando.es/adidas-originals-gazell...,Shoes,adults,adidas Originals,ash pearl/footwear white,74.95,NaN,4/9/2018,NaN


In [94]:
summary = data.describe(include='all')

In [96]:
summary.to_csv("summary.csv")

In [98]:
number_of_miss_values = pd.DataFrame(data.isnull().sum())

In [99]:
number_of_miss_values.to_csv("Missing_Values.csv")

In [68]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 177210 entries, 0 to 177209
Data columns (total 29 columns):
Unnamed: 0         177210 non-null int64
date               177210 non-null object
companyName        177210 non-null object
currency           177210 non-null object
language           177210 non-null object
countryName        177210 non-null object
distributorName    177210 non-null object
gender             164218 non-null object
modelNumber        177210 non-null object
description        176901 non-null object
productName        177210 non-null object
initialPrice       177210 non-null float64
pid                177210 non-null object
sportsCategory     124947 non-null object
type               90618 non-null object
colorGroup         170840 non-null object
group              158497 non-null object
imageUrl           177210 non-null object
articleId          177210 non-null object
consumerRating     177210 non-null float64
url                177210 non-null object
divisio

In [69]:
data = data.drop("date", axis=1)
data = data.drop("ped", axis=1)
data = data.drop("discountedSince", axis = 1)
data = data.drop("imageUrl", axis = 1)
data = data.drop("description", axis=1)
data = data.drop("articleId", axis = 1)
data = data.loc[:, ~data.columns.str.contains('^Unnamed')]

In [70]:
data_colors_na = data[data.colors.isnull()]
data_colors_na.head()

,companyName,currency,language,countryName,distributorName,gender,modelNumber,productName,initialPrice,pid,...,colorGroup,group,consumerRating,url,division,ageGroup,subBrand,colors,currentPrice,technologies
10178,adidas,EUR,es-ES,ESP,JD Sports,male,YZ000141,adidas Originals 350 Leather,90.0,1/15/2018,...,Negro,Zapatillas,0.0,https://www.jdsports.es/product/negro-adidas-o...,Shoes,adults,Sport inspired,NaN,50.0,NaN
10192,adidas,EUR,es-ES,ESP,JD Sports,male,YZ000564,adidas camiseta Outline,20.0,8/29/2017,...,Azul,Camisetas y camisetas tirantes,0.0,https://www.jdsports.es/product/azul-adidas-ca...,Clothing,adults,NaN,NaN,10.0,NaN
10195,adidas,EUR,es-ES,ESP,JD Sports,female,YZ001042,adidas UltraBoost X para mujer,180.0,4/30/2018,...,Gris,Zapatillas running,0.0,https://www.jdsports.es/product/gris-adidas-ul...,Shoes,adults,NaN,NaN,180.0,NaN
10196,adidas,EUR,es-ES,ESP,JD Sports,female,YZ001087,adidas Ultra Boost para mujer,180.0,4/23/2018,...,Blanco,Zapatillas,0.0,https://www.jdsports.es/product/blanco-adidas-...,Shoes,adults,NaN,NaN,180.0,NaN
10198,adidas,EUR,es-ES,ESP,JD Sports,male,YZ001095,adidas Originals Stan Smith Vulc,90.0,10/30/2017,...,Verde,Zapatillas skate,0.0,https://www.jdsports.es/product/verde-adidas-o...,Shoes,adults,Sport inspired,NaN,40.0,NaN


In [71]:
len(data_colors_na)

6472

In [72]:
data_colors_na_y = data_colors_na.colors
data_colors_na = data_colors_na.dropna(axis=1)

In [73]:
len(data_colors_na)

6472

In [74]:
data_colors_na.head()

,companyName,currency,language,countryName,distributorName,modelNumber,productName,initialPrice,pid,consumerRating,url,division,currentPrice
10178,adidas,EUR,es-ES,ESP,JD Sports,YZ000141,adidas Originals 350 Leather,90.0,1/15/2018,0.0,https://www.jdsports.es/product/negro-adidas-o...,Shoes,50.0
10192,adidas,EUR,es-ES,ESP,JD Sports,YZ000564,adidas camiseta Outline,20.0,8/29/2017,0.0,https://www.jdsports.es/product/azul-adidas-ca...,Clothing,10.0
10195,adidas,EUR,es-ES,ESP,JD Sports,YZ001042,adidas UltraBoost X para mujer,180.0,4/30/2018,0.0,https://www.jdsports.es/product/gris-adidas-ul...,Shoes,180.0
10196,adidas,EUR,es-ES,ESP,JD Sports,YZ001087,adidas Ultra Boost para mujer,180.0,4/23/2018,0.0,https://www.jdsports.es/product/blanco-adidas-...,Shoes,180.0
10198,adidas,EUR,es-ES,ESP,JD Sports,YZ001095,adidas Originals Stan Smith Vulc,90.0,10/30/2017,0.0,https://www.jdsports.es/product/verde-adidas-o...,Shoes,40.0


In [75]:
list(data_colors_na)

['companyName',
 'currency',
 'language',
 'countryName',
 'distributorName',
 'modelNumber',
 'productName',
 'initialPrice',
 'pid',
 'consumerRating',
 'url',
 'division',
 'currentPrice']

In [76]:
data_colors_na_train = data.loc[:, ]

In [77]:
from catboost import CatBoostClassifier
model=CatBoostClassifier(iterations=50, depth=3, learning_rate=0.1)

In [80]:
data_train_x = data.loc[:,list(data_colors_na)]

In [84]:
data_train_x.shape

(177210, 13)

In [85]:
data_train_y = data.colors

In [86]:
categorical_features_indices = np.where(data_train_x.dtypes != np.float)[0]
categorical_features_indices

array([ 0,  1,  2,  3,  4,  5,  6,  8, 10, 11], dtype=int64)

In [87]:
model.fit(data_train_x, data_train_y,cat_features=categorical_features_indices,plot=True)

ValueError: could not convert string to float: 'vapour grey/footwear white'